<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/Copy_of_Antim_Bar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# tf.config.set_visible_devices(tf.config.experimental.list_physical_devices('GPU')[0], 'GPU')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf

In [4]:
import pandas as pd

In [5]:
df=pd.read_csv('/content/drive/MyDrive/data.csv')

In [6]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


In [7]:
df.dropna(inplace=True)

In [8]:
df1=df[:3000]

In [9]:
df1.to_csv('data1.csv',index=False)

In [10]:
data=pd.read_csv('data1.csv')

In [11]:
data.shape

(3000, 6)

**Split the dataset into train and evaluate**

In [12]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(data, test_size=0.2)

In [13]:
train_data.shape

(2400, 6)

In [14]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')


In [15]:
import csv

# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [16]:
# import csv

# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [17]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.9 MB/s eta 0:00:00


In [18]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [19]:
# Load the pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [20]:
# Prepare your dataset by creating a TextDataset object
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128
)



/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [21]:
# Prepare your dataset by creating a TextDataset object
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128
)


In [22]:
# Set up the trainer with the appropriate training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    save_total_limit=2,
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset
)



In [23]:
# Fine-tune the model on your dataset
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.592800,3.346953
2,3.289700,3.309886
3,3.157100,3.297071
4,3.089000,3.296507
5,3.043300,3.298493


TrainOutput(global_step=4615, training_loss=3.218702177004334, metrics={'train_runtime': 2783.1486, 'train_samples_per_second': 26.52, 'train_steps_per_second': 1.658, 'total_flos': 4821491220480000.0, 'train_loss': 3.218702177004334, 'epoch': 5.0})

In [24]:
trainer.evaluate()

{'eval_loss': 3.298492670059204,
 'eval_runtime': 39.8979,
 'eval_samples_per_second': 92.937,
 'eval_steps_per_second': 11.63,
 'epoch': 5.0}

In [25]:
trainer

In [26]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import torch

In [28]:
# model.to('cuda')a# Convert the model to CPU

model = model.to(torch.device('cpu'))


In [29]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


which is the best phone that i can buy?) for just $99. And then, with the exception of the base model, only $34 when it comes to hardware. The iPhone 5 is coming in two models, one with $799 in cash and one with $999 in cash, but with both starting at $1,599 in cash and $999 in cash.  The iPhone SE has $999 and $999. The iPhone 12 is coming in two versions with the base model and the entry-level models with the entry-level model with the entry-level model. If you’re new to the line of iPhones and want to upgrade in price, this is your option. If you’ve made it that far in your life and have a budget, this isn’t the phone for you.  Buy your iPhone SE  and the iPhone SE   are the best and the best phones for you in the long run. The  iPhone  is not


In [30]:
# trainer.save_model('Second_Model_GPT2-Epoch_3_1000')

In [31]:
# import zipfile
# zip_ref = zipfile.ZipFile("Model_2_GPT-2_Epoch_3_1000.zip", "r")
# zip_ref.extractall()
# zip_ref.close()

In [32]:
import joblib

# joblib.dump(model, 'SecondModel_Epoch_3_1000_2.joblib')

In [33]:


joblib.dump(model, 'Model_Epoch_5_3000.joblib')

['Model_Epoch_5_3000.joblib']

In [34]:
joblib.dump(tokenizer, 'tokenizer_5_3000.joblib')


['tokenizer_5_3000.joblib']

In [31]:
loaded_model_joblib=joblib.load('SecondModel_Epoch_3_1000_2.joblib')

In [35]:
loaded_model_joblib

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
loaded_model_joblib

In [ ]:
import joblib as jb

In [ ]:
jb.dump(model,'model_jb.pkl')

In [ ]:
loaded_model = jb.load('model_jb.pkl')

In [ ]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = loaded_model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt').to('cuda'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))